In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml

In [19]:
# source=requests.get(url=input('Enter the url of the tournament \n')).text

match_url_list = []

source_matchlist = requests.get(url='https://www.vlr.gg/event/matches/800/champions-tour-north-america-stage-2-challengers/?series_id=1903').text

soup_matchlist = BeautifulSoup(source_matchlist,'lxml')

days = soup_matchlist.findAll('div', {'class':'wf-card'})

for d in range(1,len(days)):
    for i in days[d].findAll(href=True):
        if i['href'][i['href'].rindex('-')+1:] not in ('ro128', 'ro64'):
            match_url_list.append(i['href'])

In [20]:
match_url_list

['/97176/100-thieves-vs-team-reckaroo-champions-tour-north-america-stage-2-challengers-uro32',
 '/97177/t1-vs-soniqs-champions-tour-north-america-stage-2-challengers-uro32',
 '/97178/nyfu-vs-lenny-time-champions-tour-north-america-stage-2-challengers-uro32',
 '/97179/gen-g-vs-soar-champions-tour-north-america-stage-2-challengers-uro32',
 '/97180/sentinels-vs-clanless-champions-tour-north-america-stage-2-challengers-uro32',
 '/97181/akrew-vs-team-basilisk-champions-tour-north-america-stage-2-challengers-uro32',
 '/97182/built-by-gamers-academy-vs-nights-academy-champions-tour-north-america-stage-2-challengers-uro32',
 '/97183/complexity-vs-cosmic-divide-champions-tour-north-america-stage-2-challengers-uro32',
 '/97184/version1-vs-skwirel-warriers-champions-tour-north-america-stage-2-challengers-uro32',
 '/97185/girl-kissers-vs-rated-gaming-champions-tour-north-america-stage-2-challengers-uro32',
 '/97186/evil-geniuses-vs-pugstars-champions-tour-north-america-stage-2-challengers-uro32',


In [21]:
match_stats = []

for matchnum in range(len(match_url_list)):
    url = match_url_list[matchnum]
    
    source_match = requests.get(url='https://www.vlr.gg' + url).text
    
    soup_match = BeautifulSoup(source_match,'lxml')
    
    table_match = soup_match.findAll('table', {'class':'wf-table-inset mod-overview'})
    stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
    series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()
    
    for a in [a for a in list(range(len(table_match))) if a not in [2,3]]:
        if a in [0,1]:
            map_num = 1
        else:
            map_num = a//2
        
        map_name = soup_match.findAll('div', {'class':'vm-stats-gamesnav-item js-map-switch'})[map_num-1].text.strip()[1:].strip()
        stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
        team_name = soup_match.findAll('div', {'class':'wf-title-med'})[a % 2].text.strip()
        
        table = table_match[a]
        headers_match = []
        
        for i in table.find_all('th'):
            title_match = i.text.strip()
            headers_match.append(title_match)
            
        headers_match[0] = 'Player Name'
        headers_match[1] = 'Agent Name'
        headers_match[6] = 'K/D +/–'   
        headers_match[12] = 'FK/FD +/–'   
        df_match = pd.DataFrame(columns=headers_match)
        
        for row in table.find_all('tr')[1:]:
            data = row.find_all('td')
            row_data = [td.text.strip() for td in data]
            row_data[1] = str(data[1]).split('title="',1)[1].split('"',1)[0].title()
            length = len(df_match)
            df_match.loc[length] = row_data
        
        df_match['Team Name'] = team_name
        df_match['Map Name'] = map_name
        df_match['Map #'] = map_num
        df_match['Stage'] = stage
        df_match['Series'] = series
        
        match_stats.append(df_match)

In [106]:
soup_match

<!DOCTYPE html>
<html lang="en">
<head>
<title>
			Akrew vs. Sentinels | Champions Tour North America Stage 2: Challengers | Open Qualifier #2 | Valorant match | VLR.gg		</title>
<meta content="Stats, score, streams, and VODs from Akrew vs. Sentinels - Lower Round 6 match of Champions Tour North America Stage 2: Challengers Valorant event" name="description"/>
<meta charset="utf-8"/>
<meta content="Akrew vs. Sentinels | Champions Tour North America Stage 2: Challengers | Open Qualifier #2 | Valorant match | VLR.gg" property="og:title"/>
<meta content="VLR.gg" property="og:site_name"/>
<meta content="Stats, score, streams, and VODs from Akrew vs. Sentinels - Lower Round 6 match of Champions Tour North America Stage 2: Challengers Valorant event" property="og:description"/>
<meta content="/img/vlr/favicon.png" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="@vlrdotgg" name="twitter:site"/>
<meta content="Akrew vs. Sentinels | Champions Tour North Americ

In [103]:
result = pd.concat(match_stats).reset_index(drop=True)
result['Player Name'] = result['Player Name'].str.split("\n", 1).str[0].str.strip()
result['ACS'] = result['ACS'].str.split("\n", 1).str[0].str.strip() 
result['K'] = result['K'].str.split("\n", 1).str[0].str.strip()
result['D'] = result['D'].str[1:-1]
result['D'] = result['D'].str.split("\n", 3).str[2].str.strip() 
result['A'] = result['A'].str.split("\n", 1).str[0].str.strip() 
result['K/D +/–'] = result['K/D +/–'].str.split("\n", 1).str[0].str.strip() 
result['KAST'] = result['KAST'].str.split("\n", 1).str[0].str.strip().str[:-1]
result['ADR'] = result['ADR'].str.split("\n", 1).str[0].str.strip() 
result['HS%'] = result['HS%'].str.split("\n", 1).str[0].str.strip().str[:-1]
result['FK'] = result['FK'].str.split("\n", 1).str[0].str.strip() 
result['FD'] = result['FD'].str.split("\n", 1).str[0].str.strip() 
result['FK/FD +/–'] = result['FK/FD +/–'].str.split("\n", 1).str[0].str.strip() 
result = result.apply(pd.to_numeric, errors='ignore')
result['FK+FD'] = result['FK'] + result['FD']
# result.drop('KAST', axis=1, inplace = True)
result

,Player Name,Agent Name,ACS,K,D,A,K/D +/–,KAST,ADR,HS%,FK,FD,FK/FD +/–,Team Name,Map Name,Map #,Stage,Series,FK+FD
0,bang,Omen,271,18,10,8,8,74,177,35,2,4,-2,100 Thieves,Haven,1,Open Qualifier #2,Upper Round of 32,6
1,Will,Jett,266,19,13,3,6,79,157,24,6,0,6,100 Thieves,Haven,1,Open Qualifier #2,Upper Round of 32,6
2,Asuna,Skye,241,16,11,8,5,84,139,15,2,0,2,100 Thieves,Haven,1,Open Qualifier #2,Upper Round of 32,2
3,Derrek,Breach,238,18,9,8,9,79,162,31,1,0,1,100 Thieves,Haven,1,Open Qualifier #2,Upper Round of 32,1
4,stellar,Killjoy,87,3,10,9,-7,74,79,16,1,3,-2,100 Thieves,Haven,1,Open Qualifier #2,Upper Round of 32,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,TenZ,Jett,349,27,15,3,12,86,228,27,7,0,7,Sentinels,Icebox,2,Open Qualifier #2,Lower Round 6,7
1256,SicK,Sage,276,22,19,6,3,77,168,28,2,3,-1,Sentinels,Icebox,2,Open Qualifier #2,Lower Round 6,5
1257,ShahZaM,Sova,250,17,14,13,3,82,176,33,3,1,2,Sentinels,Icebox,2,Open Qualifier #2,Lower Round 6,4
1258,Kanpeki,Chamber,193,16,17,4,-1,73,125,24,3,2,1,Sentinels,Icebox,2,Open Qualifier #2,Lower Round 6,5


In [98]:
result[result['Player Name'] == 'TenZ'].head(1)

,Player Name,Agent Name,ACS,K,D,A,K/D +/–,KAST,ADR,HS%,FK,FD,FK/FD +/–,Team Name,Map Name,Map #,Stage,Series
90,TenZ,Jett,281,22,15,5,7,73,180,23,6,4,2,Sentinels,Ascent,1,Open Qualifier #2,Upper Round of 32


In [99]:
result_test = result.groupby(['Player Name']).agg({
    'ACS':'mean',
    'K':'sum',
    'D':'sum',
    'A':'sum',
    'K/D +/–':'sum',
    'KAST':'mean',
    'ADR':'mean',
    'HS%': 'mean',
    'FK':'sum',
    'FD':'sum',
    'FK/FD +/–':'sum',
    'FK+FD':'sum'})
result_test = result_test[result_test['FK+FD'] >= 50]
result_test['FB Success %'] = result_test['FK'] / result_test['FK+FD']
result_test.sort_values('FB Success %', ascending=False).head(10)

,ACS,K,D,A,K/D +/–,KAST,ADR,HS%,FK,FD,FK/FD +/–,FK+FD,FB Success %
Player Name,,,,,,,,,,,,,
TenZ,278.642857,281,189,55,92,79.285714,181.357143,26.571429,62,32,30,94,0.659574
jawgemo,295.600000,207,131,39,76,77.100000,184.800000,18.700000,37,24,13,61,0.606557
Crunchy,230.444444,133,133,31,0,70.888889,142.888889,22.111111,32,21,11,53,0.603774
koalanoob,244.000000,297,238,50,59,71.750000,155.625000,24.375000,79,52,27,131,0.603053
snirot,262.636364,222,160,31,62,69.909091,165.000000,24.545455,42,28,14,70,0.600000
ban,232.400000,198,150,24,48,72.800000,147.200000,27.700000,48,32,16,80,0.600000
ScrewFace,243.888889,158,131,32,27,75.111111,150.888889,21.222222,33,24,9,57,0.578947
rhyme,233.700000,170,147,43,23,73.900000,155.600000,27.700000,37,27,10,64,0.578125
Boostio,236.100000,169,144,33,25,70.800000,152.600000,26.300000,36,27,9,63,0.571429


In [102]:
result_test.sort_values('HS%', ascending=False).head(10)

,ACS,K,D,A,K/D +/–,KAST,ADR,HS%,FK,FD,FK/FD +/–,FK+FD,FB Success %
Player Name,,,,,,,,,,,,,
PureR,229.200000,181,142,39,39,72.700000,140.900000,36.000000,31,29,2,60,0.516667
Frosty,173.400000,134,153,53,-19,68.000000,111.600000,35.400000,26,26,0,52,0.500000
ShoT_UP,247.636364,195,173,52,22,73.545455,165.454545,31.363636,32,30,2,62,0.516129
rhyme,233.700000,170,147,43,23,73.900000,155.600000,27.700000,37,27,10,64,0.578125
ban,232.400000,198,150,24,48,72.800000,147.200000,27.700000,48,32,16,80,0.600000
Oderus,201.454545,162,155,59,7,68.636364,129.363636,27.545455,27,27,0,54,0.500000
scourge,170.111111,113,147,44,-34,64.777778,105.444444,27.444444,19,31,-12,50,0.380000
Okeanos,226.857143,126,124,22,2,66.571429,145.285714,27.142857,29,28,1,57,0.508772
TenZ,278.642857,281,189,55,92,79.285714,181.357143,26.571429,62,32,30,94,0.659574


In [104]:
result.sort_values('KAST', ascending=False).head(10)

,Player Name,Agent Name,ACS,K,D,A,K/D +/–,KAST,ADR,HS%,FK,FD,FK/FD +/–,Team Name,Map Name,Map #,Stage,Series,FK+FD
1008,Shawn,Sage,222,14,8,8,6,100,144,16,2,0,2,Gen.G,Split,2,Open Qualifier #2,Lower Round 3,2
1248,dapr,Chamber,198,10,5,3,5,100,122,29,3,0,3,Sentinels,Haven,1,Open Qualifier #2,Lower Round 6,3
1247,Kanpeki,Kayo,247,13,4,9,9,100,166,16,2,0,2,Sentinels,Haven,1,Open Qualifier #2,Lower Round 6,2
14,bang,Brimstone,159,10,6,8,4,100,109,37,1,0,1,100 Thieves,Fracture,2,Open Qualifier #2,Upper Round of 32,1
29,runi,Sova,233,11,6,8,5,100,153,27,4,0,4,Soniqs,Bind,1,Open Qualifier #2,Upper Round of 32,4
975,shonk,Kayo,327,19,8,8,11,100,233,43,1,0,1,Soniqs,Breeze,2,Open Qualifier #2,Lower Round 3,1
201,C0M,Sova,302,17,10,8,7,100,199,22,2,1,1,Evil Geniuses,Icebox,2,Open Qualifier #2,Upper Round of 32,3
110,jovahnii,Skye,298,15,7,6,8,100,189,44,2,1,1,Akrew,Bind,1,Open Qualifier #2,Upper Round of 32,3
26,dizzyLife,Brimstone,259,13,6,7,7,100,159,20,2,1,1,Soniqs,Bind,1,Open Qualifier #2,Upper Round of 32,3
25,Crunchy,Raze,341,18,10,3,8,100,197,19,4,0,4,Soniqs,Bind,1,Open Qualifier #2,Upper Round of 32,4


In [105]:
match_stats

[                                         Player Name Agent Name  \
 0  bang \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\...       Omen   
 1  Will \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\...       Jett   
 2  Asuna \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...       Skye   
 3  Derrek \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\...     Breach   
 4  stellar \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t...    Killjoy   
 
              ACS          K                   D        A     K/D +/–  \
 0  271\n177\n327  18\n4\n14  /\n\n10\n5\n5\n\n/  8\n3\n5  +8\n-1\n+9   
 1  266\n310\n241  19\n8\n11  /\n\n13\n4\n9\n\n/  3\n3\n0  +6\n+4\n+2   
 2  241\n230\n247  16\n5\n11  /\n\n11\n5\n6\n\n/  8\n4\n4   +5\n0\n+5   
 3  238\n109\n314  18\n5\n13   /\n\n9\n4\n5\n\n/  8\n0\n8  +9\n+1\n+8   
 4    87\n130\n63    3\n3\n0  /\n\n10\n5\n5\n\n/  9\n4\n5  -7\n-2\n-5   
 
             KAST            ADR            HS%       FK       FD   FK/FD +/–  \
 0  74%\n57%\n83%  177\n129\n205  35%\n44%\n31%  2\n1\n1  4\n2\n2  -2